# Support Vector Machines - SVM
## Laboratorio 3 - Diego Córdova: 20212
> En este archivo se realiza la limpieza de datos, clasificacion y se describen los hallazgos y Conclusiones

## [Fraude en el Vino](https://en.wikipedia.org/wiki/Wine_fraud)

El fraude en el vino se relaciona a los aspectos comerciales del vino.  El tipo más prevalente de fraude es aquel en el cual los vinos son adulterados, generalmente con la adición de productos más baratos (e.g. jugos) y, a veces, con químicos dañinos y endulcorantes (para compensar el color o sabor).

La falsificación y la re-etiquetación de vinos inferiores y más baratos haciéndolos parecer marcas más caras es otro tipo común de fraude con vino.

<img src="wine.jpg">

## Objetivos del Laboratorio

Una compañía distribuidora, que recientemente fue víctima de fraude, ha completado una auditoría de varias muestras de vino a través del uso de análisis químico.  La compañía se especializa en la exportación de vinos de una calidad extremadamente alta y muy valiosos.  El fraude fue cometido por un proveedor que intentó pasar vinos baratos, de calidad inferior, como vinos de más alta gama.  La compañía distribuidora los ha contratado a Ustedes para intentar crear un modelo de aprendizaje de máquina (ML) que pueda ayudar a detectar muestras de vino de calidad baja ("fraude").  Quieren saber si siquiera es posible detectar tal diferencia.  


Fuente de datos: *P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553, 2009.*

---
---

**La meta general es utilizar el conjunto de datos adjunto para desarrollar un modelo que intenta predecir si un vino es legítimo "Legit" o fraudulento "Fraud" basado en varias características químicas.**

---
---

**Rubrica**
* Limpieza de datos

**(36 puntos) Clasificación**
* Realiza varios modelos de Clasificación, analiza los resultados para identificar el de mejor rendimiento
        
**(14 puntos) Hallazgos y conclusiones.**
* Hace la recomendación correcta para la Compañía Distribuidora

**NOTA:  Al usar SVC, anteriormente hemos dejado el modelo en blanco, (e.g. sin parámeteros).  Sin embargo, en estos datos las clases no están balanceadas.  Para intentar remediar este problema, podemos ajustar automáticamente los pesos de forma inversamente proporcional a las frecuencias de las clases en los datos con una llamada en la invocación de SVC().  Ver la [documentation for SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) en-línea y averiguen cuál es el argumento\parámetro.**

### Limpieza de Datos

In [1]:
# Imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('./wine_fraud.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Legit,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Legit,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Legit,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Legit,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Legit,red


Se convierte la variable "free sulfur dioxide" a entero

In [3]:
data['free sulfur dioxide'] = data['free sulfur dioxide'].astype(int)
data[['free sulfur dioxide']].head()

,free sulfur dioxide
0,11
1,25
2,15
3,17
4,11


Se elimina la variable type

In [4]:
data = data.drop(columns=['type'], axis=1)
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11,34.0,0.9978,3.51,0.56,9.4,Legit
1,7.8,0.88,0.00,2.6,0.098,25,67.0,0.9968,3.20,0.68,9.8,Legit
2,7.8,0.76,0.04,2.3,0.092,15,54.0,0.9970,3.26,0.65,9.8,Legit
3,11.2,0.28,0.56,1.9,0.075,17,60.0,0.9980,3.16,0.58,9.8,Legit
4,7.4,0.70,0.00,1.9,0.076,11,34.0,0.9978,3.51,0.56,9.4,Legit


Encoding de varibles categoricas

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['quality'] = encoder.fit_transform(data['quality'])
data['quality'].unique()

array([1, 0])

In [6]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11,34.0,0.99780,3.51,0.56,9.4,1
1,7.8,0.88,0.00,2.6,0.098,25,67.0,0.99680,3.20,0.68,9.8,1
2,7.8,0.76,0.04,2.3,0.092,15,54.0,0.99700,3.26,0.65,9.8,1
3,11.2,0.28,0.56,1.9,0.075,17,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11,34.0,0.99780,3.51,0.56,9.4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24,92.0,0.99114,3.27,0.50,11.2,1
6493,6.6,0.32,0.36,8.0,0.047,57,168.0,0.99490,3.15,0.46,9.6,1
6494,6.5,0.24,0.19,1.2,0.041,30,111.0,0.99254,2.99,0.46,9.4,1
6495,5.5,0.29,0.30,1.1,0.022,20,110.0,0.98869,3.34,0.38,12.8,1


### Clasificacion

In [7]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.05, random_state = 0)

Regresion Logistica

In [28]:
# Balanceo
from imblearn.over_sampling import RandomOverSampler
oversample = RandomOverSampler(random_state=42)
X_entreno, y_entreno = oversample.fit_resample(X_entreno, y_entreno)

# Entrenar Modelo
from sklearn.linear_model import LogisticRegression
modeloLog = LogisticRegression(max_iter = 500)
modeloLog.fit(X_entreno,y_entreno)
predicciones = modeloLog.predict(X_prueba)

# Mediciones
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_prueba, predicciones))
print(confusion_matrix(y_prueba, predicciones))
accuracy_score(y_prueba, predicciones)

              precision    recall  f1-score   support

           0       0.11      0.71      0.19        14
           1       0.98      0.74      0.84       311

    accuracy                           0.74       325
   macro avg       0.55      0.73      0.52       325
weighted avg       0.95      0.74      0.82       325

[[ 10   4]
 [ 81 230]]


0.7384615384615385

KNN

In [29]:
# Entreno de Modelo
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(X_entreno, y_entreno)
pred = knn.predict(X_prueba)

# Evaluacion del modelo
print(classification_report(y_prueba, pred))
print(confusion_matrix(y_prueba, pred))
accuracy_score(y_prueba, y_pred)


              precision    recall  f1-score   support

           0       0.40      0.43      0.41        14
           1       0.97      0.97      0.97       311

    accuracy                           0.95       325
   macro avg       0.69      0.70      0.69       325
weighted avg       0.95      0.95      0.95       325

[[  6   8]
 [  9 302]]


0.8584615384615385

SVMs

In [30]:
from sklearn.svm import SVC
clasificador = SVC(kernel = 'linear', random_state = 0, class_weight='balanced')
clasificador.fit(X_entreno, y_entreno)

y_pred = clasificador.predict(X_prueba)
print(classification_report(y_prueba, y_pred))
print(confusion_matrix(y_prueba, y_pred))
accuracy_score(y_prueba, y_pred)

              precision    recall  f1-score   support

           0       0.11      0.64      0.19        14
           1       0.98      0.76      0.86       311

    accuracy                           0.76       325
   macro avg       0.54      0.70      0.52       325
weighted avg       0.94      0.76      0.83       325

[[  9   5]
 [ 74 237]]


0.7569230769230769

Kernel SVM

In [27]:
from sklearn.preprocessing import StandardScaler
normalizador = StandardScaler()
X_entreno = normalizador.fit_transform(X_entreno)
X_prueba = normalizador.transform(X_prueba)

from sklearn.svm import SVC
clasificador = SVC(kernel = 'rbf', random_state = 0, class_weight='balanced')
clasificador.fit(X_entreno, y_entreno)

y_pred = clasificador.predict(X_prueba)
from sklearn.metrics import confusion_matrix, accuracy_score
mat_conf = confusion_matrix(y_prueba, y_pred)
print(mat_conf)
accuracy_score(y_prueba, y_pred)

[[ 11   3]
 [ 43 268]]


0.8584615384615385

Naive Bayes

### Hallazgos y Conclusiones